In [1]:
import os
os.getcwd()

'/home/bhvignesh/trading_system/notebooks'

In [2]:
os.chdir('..')

In [3]:
from src.optimizer.strategy_optimizer import StrategyOptimizer
from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
from src.strategies.momentum.awesome_oscillator_strat import AwesomeOscillatorStrategy
from src.database.config import DatabaseConfig
from hyperopt import hp
import mlflow

In [4]:
db_config = DatabaseConfig.default()

In [5]:
mlflow.set_tracking_uri("file:/home/bhvignesh/trading_system/mlruns")

In [6]:
import src.collectors.data_collector as data_collector
import pandas as pd

tickers = pd.read_excel("data/ticker.xlsx")
tickers = tickers[~tickers["Security Name"].duplicated()]
tickers.reset_index(drop=True, inplace=True)

def add_ticker_suffix(x):
    if x["Exchange"]=="BSE":
        return x["Security Name"] + ".BO"
    else:
        return x["Security Name"] + ".NS"

tickers["Ticker"] = tickers.apply(add_ticker_suffix, axis = 1)
all_tickers = tickers["Ticker"].tolist()

In [7]:
from src.optimizer.search_space import awesome_oscillator_strat_search_space

In [8]:
tickers = all_tickers[:10] 

In [ ]:
with mlflow.start_run(run_name="UsageExample"):
    # Initialize the optimizer.
    optimizer = StrategyOptimizer(
        strategy_class=AwesomeOscillatorStrategy,  # Replace with your strategy
        db_config=db_config,
        search_space=awesome_oscillator_strat_search_space,
        tickers=tickers,
        start_date="2020-01-01",
        end_date="2022-12-31",
        max_evals=50
    )

    # Run the hyperparameter optimization.
    best_params, performance_report, param_history = optimizer.run_optimization()
    print("Best Parameters:")
    print(best_params)
    print("\nPerformance Report:")
    print(performance_report)

Exception: Run with UUID 1976cda82be34f4aa09dd0b03be307bd is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [10]:
# Ensure any active run is ended before starting a new one
if mlflow.active_run():
    mlflow.end_run()

In [17]:
mlflow.end_run()